# 🎾 Tennis GOAT Project – ETL Data Preparation

>Ce notebook contient l'étape de préparation des données (ETL) à partir du dataset ATP fourni par Jeff Sackmann (1968–2024).

🎯 **Objectif : construire une base propre pour analyser les performances des joueurs et identifier le GOAT et ses potentiels successeurs.**

# Import des Libraries

In [2]:
import pandas as pd
import os
import glob
import warnings
warnings.filterwarnings("ignore")  

##  Chargement des fichiers atp_matches_*.csv et fusion en un seul DF

In [3]:
data_path = r"C:\Users\jackd\Downloads\Portfolio Data Analyst\Python\Tennis\data_tennis"
all_files = glob.glob(os.path.join(data_path, "atp_matches_*.csv"))

print(f"{len(all_files)} fichiers trouvés")

160 fichiers trouvés


In [4]:
df_list = [pd.read_csv(f) for f in all_files]
df_all = pd.concat(df_list, ignore_index=True)
print(f"{df_all.shape[0]} lignes au total après fusion")

968048 lignes au total après fusion


# Conversion de date et d'une colonne pour l'année

In [5]:
df_all['tourney_date'] = pd.to_datetime(df_all['tourney_date'], format='%Y%m%d', errors='coerce')
df_all['year'] = df_all['tourney_date'].dt.year

# Nettoyage des doublons

In [6]:
print(f"Doublons exacts : {df_all.duplicated().sum()}")
df_all = df_all.drop_duplicates()

Doublons exacts : 3


# Standardisation des noms (si espace par ex)

In [7]:
df_all['winner_name'] = df_all['winner_name'].str.strip()
df_all['loser_name'] = df_all['loser_name'].str.strip()

# Supression de lignes incomplètes (possibilité pas de perdant/gagnant mais très rare)

In [8]:
df_all = df_all[df_all['winner_name'].notnull() & df_all['loser_name'].notnull()]
df_all = df_all[df_all['winner_name'] != '']
df_all = df_all[df_all['loser_name'] != '']

# Sélection des colonnes qui seront utiles aux analyses suivantes

In [9]:
cols_to_keep = [
    'tourney_id', 'tourney_name', 'surface', 'tourney_level', 'tourney_date', 'year',
    'winner_name', 'loser_name', 'score', 'round',
    'winner_rank', 'loser_rank', 'winner_seed', 'loser_seed'
]
df_all = df_all[cols_to_keep]

# Nettoyage des types numériques
df_all['winner_rank'] = pd.to_numeric(df_all['winner_rank'], errors='coerce')
df_all['loser_rank'] = pd.to_numeric(df_all['loser_rank'], errors='coerce')
df_all['winner_seed'] = pd.to_numeric(df_all['winner_seed'], errors='coerce')
df_all['loser_seed'] = pd.to_numeric(df_all['loser_seed'], errors='coerce')

# Standardisation des surfaces

In [10]:
df_all['surface'] = df_all['surface'].str.lower()
df_all = df_all[df_all['surface'].isin(['clay', 'hard', 'grass', 'carpet'])]

# Double Check

In [11]:
print(df_all.shape)
df_all.head()
df_all.info()

(937012, 14)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 937012 entries, 0 to 968047
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   tourney_id     937012 non-null  object        
 1   tourney_name   937012 non-null  object        
 2   surface        937012 non-null  object        
 3   tourney_level  937012 non-null  object        
 4   tourney_date   937012 non-null  datetime64[ns]
 5   year           937012 non-null  float64       
 6   winner_name    937012 non-null  object        
 7   loser_name     937012 non-null  object        
 8   score          936858 non-null  object        
 9   round          937012 non-null  object        
 10  winner_rank    832144 non-null  float64       
 11  loser_rank     763597 non-null  float64       
 12  winner_seed    399776 non-null  float64       
 13  loser_seed     205007 non-null  float64       
dtypes: datetime64[ns](1), float64(5), objec

# Save

In [12]:
output_path = r"C:\Users\jackd\Downloads\Portfolio Data Analyst\Python\Tennis\all_matches_clean.csv"
df_all.to_csv(output_path, index=False)
print("Fichier sauvegardé avec succès.")

Fichier sauvegardé avec succès.
